# Init

In [1]:
%%capture
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
import re
from unidecode import unidecode

2023-02-03 15:56:19.816402: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-03 15:56:20.246524: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-03 15:56:20.246551: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-03 15:56:21.404287: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-0

In [2]:
data = pd.read_csv("train_ner.csv")
data.head()

,id,text,label
0,32674,da de unde stii u mai [ORG] ca banii au fost p...,ABUSE
1,16514,m*uie [PERS] m*uie [PERS] ... m*uie\nbai kakat...,INSULT
2,32556,PT ALA CARE ARE TREABA CU [PERS]!!ESTI UNUL CA...,OTHER
3,23861,sunt bucuros ca [PERS] nu a mai venit la [ORG]...,OTHER
4,21811,[PERS] esti....PE..N\n ES..CU..LI..BI.L!!! te ...,INSULT


In [3]:
import spacy

2023-02-03 15:56:25.485985: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-03 15:56:25.486261: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-03 15:56:25.486282: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (andrei-VirtualBox): /proc/driver/nvidia/version does not exist


In [4]:
nlp = spacy.load('ro_core_news_lg')


In [5]:
stops = nlp.Defaults.stop_words

In [6]:
def preprocess(x):
    s = unidecode(x)
    s = str.lower(s)
    s = re.sub(r"\[[a-z]+\]","", s)
    s = re.sub(r"\*","", s)
    s = re.sub(r"[^a-zA-Z0-9]+"," ",s)
    s = re.sub(r" +"," ",s)
    s = re.sub(r"(.)\1+",r"\1",s)
    s = " ".join([elem for elem in s.split(' ') if elem not in stops])
    return s

data["preprocessed"] = data["text"].map(preprocess)
data.head(5)

,id,text,label,preprocessed
0,32674,da de unde stii u mai [ORG] ca banii au fost p...,ABUSE,sti bani pt pt dulce scumpa bomboana adik pron...
1,16514,m*uie [PERS] m*uie [PERS] ... m*uie\nbai kakat...,INSULT,muie muie muie bai kakatule stai banca arbitru...
2,32556,PT ALA CARE ARE TREABA CU [PERS]!!ESTI UNUL CA...,OTHER,pt treaba esti ide fotbal
3,23861,sunt bucuros ca [PERS] nu a mai venit la [ORG]...,OTHER,bucuros venit jucator duzina spere echipa nati...
4,21811,[PERS] esti....PE..N\n ES..CU..LI..BI.L!!! te ...,INSULT,esti es bi asemeni chiajna parveniti fotbal g...


In [7]:
label_ids = {label_name:i for i, label_name in enumerate(sorted(set(data["label"])))}
label_ids

{'ABUSE': 0, 'INSULT': 1, 'OTHER': 2, 'PROFANITY': 3}

In [8]:
l = [elem.split(" ") for elem in data["preprocessed"].tolist()]
s = set()
for elem in l:
    s = s|set(elem)
print(len(s))

27240


In [9]:
data["class"] = data["label"].map(lambda x: label_ids[x])
data.head(5)

,id,text,label,preprocessed,class
0,32674,da de unde stii u mai [ORG] ca banii au fost p...,ABUSE,sti bani pt pt dulce scumpa bomboana adik pron...,0
1,16514,m*uie [PERS] m*uie [PERS] ... m*uie\nbai kakat...,INSULT,muie muie muie bai kakatule stai banca arbitru...,1
2,32556,PT ALA CARE ARE TREABA CU [PERS]!!ESTI UNUL CA...,OTHER,pt treaba esti ide fotbal,2
3,23861,sunt bucuros ca [PERS] nu a mai venit la [ORG]...,OTHER,bucuros venit jucator duzina spere echipa nati...,2
4,21811,[PERS] esti....PE..N\n ES..CU..LI..BI.L!!! te ...,INSULT,esti es bi asemeni chiajna parveniti fotbal g...,1


In [10]:
MAX_FEATURES = 10000
SEQUENCE_LENGTH = 70
BATCH_SIZE = 32
EMBEDDING_DIM = 400
EPOCHS = 10

In [11]:
def prepare_dataset(filename):
    df = pd.read_csv(filename)
    df["preprocessed"] = df["text"].map(preprocess)
    labels = df["label"].map(lambda x: label_ids[x]).apply(lambda x: tf.keras.utils.to_categorical(int(x), len(label_ids))).values.tolist()
    res_ds = tf.data.Dataset.from_tensor_slices((df["preprocessed"],labels)).shuffle(len(df)).batch(BATCH_SIZE)
    return res_ds

In [12]:
train_ds = prepare_dataset("train_ner.csv")
test_ds = prepare_dataset("test.csv")
val_ds = prepare_dataset("validation_internal_ner.csv")

2023-02-03 15:56:41.332634: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
texts = train_ds.map(lambda x,y:x)
vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=MAX_FEATURES,
    output_mode="int",
    output_sequence_length=SEQUENCE_LENGTH,
    standardize=None,
    split="whitespace",
)
vectorizer.adapt(texts)

In [14]:
input = tf.keras.Input(shape=(),dtype=tf.string)
tokens = vectorizer(input)
embed_layer = tf.keras.layers.Embedding(
    input_dim=MAX_FEATURES,
    output_dim=EMBEDDING_DIM,
    mask_zero=True
)
embeds = embed_layer(tokens)
rnn_layer = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(
        units=10
    )
)
# rnn_layer2 = tf.keras.layers.Bidirectional(
#     tf.keras.layers.LSTM(
#         units=10
#     ),
#     merge_mode="ave"
# )

h = rnn_layer(embeds)
# h = rnn_layer2(h)
# fc1 = tf.keras.layers.Dense(10,activation="relu")
fc2 = tf.keras.layers.Dense(len(label_ids),activation='softmax')

# h = fc1(h)
# h = tf.keras.layers.Dropout(0.2)(h)
output = fc2(h)

model = tf.keras.Model(inputs=input,outputs=output)

In [16]:
cback = tf.keras.callbacks.EarlyStopping(
    monitor='val_f1_micro',
    min_delta=0,
    patience=2,
    mode='max',
    verbose=2,
    baseline=None,
    restore_best_weights=True
)
f1_metric_micro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='micro', name='f1_micro')
f1_metric_macro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='macro', name='f1_macro')
optim = tfa.optimizers.AdamW(weight_decay=0.001,learning_rate=0.001)
model.compile(
    optimizer=optim,
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[
        tf.keras.metrics.CategoricalAccuracy(),
        tf.keras.metrics.Precision(name='precison'),
        tf.keras.metrics.Recall(name='recall'),
        f1_metric_micro,
        f1_metric_macro
    ],
)

In [17]:
model.fit(x=train_ds, epochs=10, validation_data=val_ds, callbacks=[cback])

Epoch 1/10
312/312 [==============================] - 34s 91ms/step - loss: 1.1059 - categorical_accuracy: 0.5350 - precison: 0.7147 - recall: 0.2383 - f1_micro: 0.3574 - f1_macro: 0.2620 - val_loss: 0.6552 - val_categorical_accuracy: 0.8152 - val_precison: 0.9143 - val_recall: 0.6163 - val_f1_micro: 0.7363 - val_f1_macro: 0.6480
Epoch 2/10
312/312 [==============================] - 27s 88ms/step - loss: 0.5638 - categorical_accuracy: 0.8051 - precison: 0.8606 - recall: 0.7277 - f1_micro: 0.7886 - f1_macro: 0.7704 - val_loss: 0.2883 - val_categorical_accuracy: 0.9277 - val_precison: 0.9516 - val_recall: 0.9076 - val_f1_micro: 0.9290 - val_f1_macro: 0.9220
Epoch 3/10
312/312 [==============================] - 29s 93ms/step - loss: 0.2845 - categorical_accuracy: 0.9164 - precison: 0.9372 - recall: 0.8931 - f1_micro: 0.9146 - f1_macro: 0.9095 - val_loss: 0.1593 - val_categorical_accuracy: 0.9648 - val_precison: 0.9759 - val_recall: 0.9548 - val_f1_micro: 0.9652 - val_f1_macro: 0.9621
Epoc

In [18]:
model.evaluate(test_ds)

78/78 [==============================] - 1s 11ms/step - loss: 1.1504 - categorical_accuracy: 0.6425 - precison: 0.6577 - recall: 0.6260 - f1_micro: 0.6414 - f1_macro: 0.6344


[1.1504476070404053,
 0.6424558758735657,
 0.6576728224754333,
 0.6260032057762146,
 0.6414473652839661,
 0.6343844532966614]

In [19]:
test_ds2 = prepare_dataset("test_ner.csv")

In [20]:
model.evaluate(test_ds2)

78/78 [==============================] - 1s 14ms/step - loss: 1.0537 - categorical_accuracy: 0.6645 - precison: 0.6843 - recall: 0.6437 - f1_micro: 0.6634 - f1_macro: 0.6561


[1.0537015199661255,
 0.6645264625549316,
 0.6843003630638123,
 0.6436597108840942,
 0.6633581519126892,
 0.6560677289962769]

In [25]:
m = data["label"].value_counts().values

In [26]:
import numpy as np

In [30]:
m/min(m)

array([2.81993818, 2.13910355, 1.73261206, 1.        ])

In [31]:
test_data = pd.read_csv("test_ner.csv")
m2 = test_data["label"].value_counts().values
m2 / min(m2)

array([2.71299094, 2.06646526, 1.74924471, 1.        ])

In [35]:
lengths = np.array([len(elem.split(" ")) for elem in data["preprocessed"]])

In [42]:
print(np.mean(lengths), np.std(lengths))

17.273384909072643 10.580427950116055


In [37]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Histogram(x=lengths))
fig.show()